# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2022-11-29'
start_date = '2016-01-01'
# Descargar el dataframe
ibex_df = yf.download('^IBEX',start_date, today).reset_index()

ibex_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1763,2022-11-22,"$8,203.50","$8,345.60","$8,200.30","$8,325.00","$8,325.00",215622700
1764,2022-11-23,"$8,337.80","$8,344.00","$8,288.40","$8,331.20","$8,331.20",154543800
1765,2022-11-24,"$8,341.30","$8,412.50","$8,339.20","$8,388.10","$8,388.10",137787400
1766,2022-11-25,"$8,391.20","$8,425.50","$8,378.20","$8,416.60","$8,416.60",135912400
1767,2022-11-28,"$8,375.60","$8,396.40","$8,316.70","$8,323.20","$8,323.20",125888100


In [ ]:
# Validamos que no hay datos vacíos
ibex_df.isnull().sum()

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del Ibex en ese momento
df = ibex_df[['Date', 'Open']]

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
new_names = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df
df = df.rename(new_names,axis=1)
display(df)


,ds,y
0,2016-01-04,"$9,429.80"
1,2016-01-05,"$9,431.50"
2,2016-01-06,"$9,330.30"
3,2016-01-07,"$8,973.80"
4,2016-01-08,"$9,093.30"
...,...,...
1763,2022-11-22,"$8,203.50"
1764,2022-11-23,"$8,337.80"
1765,2022-11-24,"$8,341.30"
1766,2022-11-25,"$8,391.20"


In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1763,2022-11-22,"$8,203.50"
1764,2022-11-23,"$8,337.80"
1765,2022-11-24,"$8,341.30"
1766,2022-11-25,"$8,391.20"
1767,2022-11-28,"$8,375.60"


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Precio de apertura IBEX"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet(seasonality_mode='multiplicative')

In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2128,2023-11-24
2129,2023-11-25
2130,2023-11-26
2131,2023-11-27
2132,2023-11-28


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
2126,2023-11-22
2127,2023-11-23
2128,2023-11-24
2131,2023-11-27


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)
forecast

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2016-01-04,"$1,758.74","$8,224.67","$8,988.73","$1,758.74","$1,758.74",$3.89,$3.89,$3.89,$3.83,$3.83,$3.83,$0.06,$0.06,$0.06,$0.00,$0.00,$0.00,"$8,591.81"
1,2016-01-05,"$1,759.20","$8,229.97","$9,005.38","$1,759.20","$1,759.20",$3.90,$3.90,$3.90,$3.83,$3.83,$3.83,$0.07,$0.07,$0.07,$0.00,$0.00,$0.00,"$8,614.22"
2,2016-01-06,"$1,759.65","$8,250.04","$9,015.75","$1,759.65","$1,759.65",$3.91,$3.91,$3.91,$3.83,$3.83,$3.83,$0.07,$0.07,$0.07,$0.00,$0.00,$0.00,"$8,633.05"
3,2016-01-07,"$1,760.11","$8,265.77","$9,012.48","$1,760.11","$1,760.11",$3.91,$3.91,$3.91,$3.83,$3.83,$3.83,$0.08,$0.08,$0.08,$0.00,$0.00,$0.00,"$8,645.80"
4,2016-01-08,"$1,760.57","$8,279.70","$9,049.76","$1,760.57","$1,760.57",$3.92,$3.92,$3.92,$3.83,$3.83,$3.83,$0.09,$0.09,$0.09,$0.00,$0.00,$0.00,"$8,655.20"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024,2023-11-22,"$1,463.34","$3,155.69","$10,561.59",$654.46,"$2,194.10",$3.81,$3.81,$3.81,$3.83,$3.83,$3.83,$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,"$7,033.27"
2025,2023-11-23,"$1,462.86","$3,097.19","$10,585.15",$650.66,"$2,197.02",$3.80,$3.80,$3.80,$3.83,$3.83,$3.83,$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,"$7,026.62"
2026,2023-11-24,"$1,462.39","$3,092.77","$10,490.87",$647.57,"$2,199.75",$3.80,$3.80,$3.80,$3.83,$3.83,$3.83,$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,"$7,017.96"
2027,2023-11-27,"$1,460.96","$2,991.65","$10,565.91",$638.59,"$2,207.75",$3.80,$3.80,$3.80,$3.83,$3.83,$3.83,$-0.03,$-0.03,$-0.03,$0.00,$0.00,$0.00,"$7,006.57"


In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2024,2023-11-22,"$7,033.27","$3,155.69","$10,561.59"
2025,2023-11-23,"$7,026.62","$3,097.19","$10,585.15"
2026,2023-11-24,"$7,017.96","$3,092.77","$10,490.87"
2027,2023-11-27,"$7,006.57","$2,991.65","$10,565.91"
2028,2023-11-28,"$7,007.01","$3,173.34","$10,562.99"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predicho.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)